In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.dates as md
import seaborn as sns
import datetime as datetime
import numpy as np
import os

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid")

import warnings 
warnings.filterwarnings('ignore')

my_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

Limpieza_Clicks = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Clicks.ipynb'))
Limpieza_Auctions = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Auctions.ipynb'))
Limpieza_Installs = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Installs.ipynb'))
Limpieza_Events = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Events.ipynb'))

In [3]:
%run $Limpieza_Clicks
%run $Limpieza_Auctions
%run $Limpieza_Installs
%run $Limpieza_Events

# Cálculo del St

In [4]:
training_set = auctions
training_set.sort_values(ascending=True,by='date',inplace=True)

In [5]:
#Separamos el set de datos en "ventanas" de 3 días cada una.
ventana1 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-18')) & (training_set['date'] < pd.to_datetime('2019-04-21'))]
ventana2 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-19')) & (training_set['date'] < pd.to_datetime('2019-04-22'))]
ventana3 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-20')) & (training_set['date'] < pd.to_datetime('2019-04-23'))]
ventana4 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-21')) & (training_set['date'] < pd.to_datetime('2019-04-24'))]
ventana5 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-22')) & (training_set['date'] < pd.to_datetime('2019-04-25'))]
ventana6 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-23')) & (training_set['date'] < pd.to_datetime('2019-04-26'))]
ventana7 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-24')) & (training_set['date'] < pd.to_datetime('2019-04-27'))]

Para cada ventana obtenemos la primer y la última subasta en que participo cada dispositivo

In [6]:
ventana1['rg_min'] = ventana1.groupby('device_id')['date'].transform(min)
ventana1['rg_max'] = ventana1.groupby('device_id')['date'].transform(max)

train_st = ventana1[['device_id','rg_min','rg_max']].drop_duplicates()
train_st.columns = ['device_id','v1_min','v1_max']

In [7]:
ventana2['rg_min'] = ventana2.groupby('device_id')['date'].transform(min)
ventana2['rg_max'] = ventana2.groupby('device_id')['date'].transform(max)

train_st = pd.merge(train_st,ventana2[['device_id','rg_min','rg_max']].drop_duplicates(),on='device_id',how='outer')
train_st.columns = ['device_id','v1_min','v1_max','v2_min','v2_max']

In [8]:
ventana3['rg_min'] = ventana3.groupby('device_id')['date'].transform(min)
ventana3['rg_max'] = ventana3.groupby('device_id')['date'].transform(max)

train_st = pd.merge(train_st,ventana3[['device_id','rg_min','rg_max']].drop_duplicates(),on='device_id',how='outer')
train_st.columns = ['device_id','v1_min','v1_max','v2_min','v2_max','v3_min','v3_max']

In [9]:
ventana4['rg_min'] = ventana4.groupby('device_id')['date'].transform(min)
ventana4['rg_max'] = ventana4.groupby('device_id')['date'].transform(max)

train_st = pd.merge(train_st,ventana4[['device_id','rg_min','rg_max']].drop_duplicates(),on='device_id',how='outer')
train_st.columns = ['device_id','v1_min','v1_max','v2_min','v2_max','v3_min','v3_max','v4_min','v4_max']

In [10]:
ventana5['rg_min'] = ventana5.groupby('device_id')['date'].transform(min)
ventana5['rg_max'] = ventana5.groupby('device_id')['date'].transform(max)

train_st = pd.merge(train_st,ventana5[['device_id','rg_min','rg_max']].drop_duplicates(),on='device_id',how='outer')
train_st.columns = ['device_id','v1_min','v1_max','v2_min','v2_max','v3_min','v3_max','v4_min','v4_max','v5_min','v5_max']

In [11]:
ventana6['rg_min'] = ventana6.groupby('device_id')['date'].transform(min)
ventana6['rg_max'] = ventana6.groupby('device_id')['date'].transform(max)

train_st = pd.merge(train_st,ventana6[['device_id','rg_min','rg_max']].drop_duplicates(),on='device_id',how='outer')
train_st.columns = ['device_id','v1_min','v1_max','v2_min','v2_max','v3_min','v3_max','v4_min','v4_max','v5_min','v5_max','v6_min','v6_max']

In [12]:
ventana7['rg_min'] = ventana7.groupby('device_id')['date'].transform(min)
ventana7['rg_max'] = ventana7.groupby('device_id')['date'].transform(max)

train_st = pd.merge(train_st,ventana7[['device_id','rg_min','rg_max']].drop_duplicates(),on='device_id',how='outer')
train_st.columns = ['device_id','v1_min','v1_max','v2_min','v2_max','v3_min','v3_max','v4_min','v4_max','v5_min','v5_max','v6_min','v6_max','v7_min','v7_max']

Calculamos el St por ventana como la diferencia de tiempo (en segundos) entre la última subasta de una ventana con la primera subasta en los 3 días siguientes

In [13]:
train_st['st_v1'] =  abs((train_st['v1_max'] - train_st['v4_min']).dt.total_seconds())
train_st['st_v2'] =  abs((train_st['v2_max'] - train_st['v5_min']).dt.total_seconds())
train_st['st_v3'] =  abs((train_st['v3_max'] - train_st['v6_min']).dt.total_seconds())
train_st['st_v4'] =  abs((train_st['v4_max'] - train_st['v7_min']).dt.total_seconds())

In [14]:
train_st = train_st[['device_id','st_v1','st_v2','st_v3','st_v4']]

In [15]:
train_st.head()

,device_id,st_v1,st_v2,st_v3,st_v4
0,1826643666390887030,94930.370193,100941.615377,100941.615377,216859.798358
1,7037174172278258682,219.606447,3763.735472,5334.823385,9475.648466
2,3392065368947589877,347185.086787,347185.086787,NaN,67974.352169
3,1228982273563226229,13280.234792,107721.277293,1735.126772,35732.474128
4,4123059034628125459,399399.762981,NaN,78072.495218,5002.646882


En caso de que no haya habido conversión para alguna ventana consideramos a St como el tiempo máximo (3 días en segundos)

In [16]:
tiempo_maximo = 259200
train_st.fillna(tiempo_maximo,inplace=True)

In [17]:
train_st.head()

,device_id,st_v1,st_v2,st_v3,st_v4
0,1826643666390887030,94930.370193,100941.615377,100941.615377,216859.798358
1,7037174172278258682,219.606447,3763.735472,5334.823385,9475.648466
2,3392065368947589877,347185.086787,347185.086787,259200.000000,67974.352169
3,1228982273563226229,13280.234792,107721.277293,1735.126772,35732.474128
4,4123059034628125459,399399.762981,259200.000000,78072.495218,5002.646882


# FEATURES

Creamos features en base a los datasets. Los features van a estar divididos por dispositivo para poder mergearlos en un unico dataframe con features de todos los dataframes.

### Features de Auctions

In [18]:
features_auctions = auctions[['device_id','ref_type_id']].drop_duplicates()

In [19]:
features_auctions.sort_values('device_id',inplace=True)
features_auctions.columns = ['device_id','es_Apple_o_Google']

In [20]:
#No sirve porque hay dispositivos que usan ambas...
features_auctions

,device_id,es_Apple_o_Google
36601349,40621409780134,Google
30900602,41863526108385,Google
39675750,69039685746313,Google
39028251,135153013040192,Google
46872336,161514654074162,Google
33917684,168103949904656,Google
10607289,181891380775191,Google
16127540,186034136943920,Google
293739,283297668933729,Google
13736755,295841792051458,Google


In [52]:
auctions_v1 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-18')) & (auctions['date'] < pd.to_datetime('2019-04-21'))]
auctions_v2 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-19')) & (auctions['date'] < pd.to_datetime('2019-04-22'))]
auctions_v3 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-20')) & (auctions['date'] < pd.to_datetime('2019-04-23'))]
auctions_v4 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-21')) & (auctions['date'] < pd.to_datetime('2019-04-24'))]
auctions_v5 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-22')) & (auctions['date'] < pd.to_datetime('2019-04-25'))]

features_auctions_v1 = auctions_v1.groupby('device_id').agg({'device_id':'size'})
features_auctions_v2 = auctions_v2.groupby('device_id').agg({'device_id':'size'})
features_auctions_v3 = auctions_v3.groupby('device_id').agg({'device_id':'size'})
features_auctions_v4 = auctions_v4.groupby('device_id').agg({'device_id':'size'})
features_auctions_v5 = auctions_v5.groupby('device_id').agg({'device_id':'size'})

In [53]:
features_auctions_v1.columns = ['cantidad_de_subastas_en_ventana']
features_auctions_v2.columns = ['cantidad_de_subastas_en_ventana']
features_auctions_v3.columns = ['cantidad_de_subastas_en_ventana']
features_auctions_v4.columns = ['cantidad_de_subastas_en_ventana']
features_auctions_v5.columns = ['cantidad_de_subastas_en_ventana']

In [54]:
Week_Day = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

for dia in Week_Day:
    subastas_por_dia = auctions[auctions['date'].dt.day_name() == dia].groupby('device_id').agg({'device_id':'size'})
    subastas_por_dia.columns = ['cantidad_eventos_{}'.format(dia)]
    
    features_auctions_v1 = features_auctions_v1.merge(subastas_por_dia, how='outer', left_index=True, right_index=True)
    features_auctions_v2 = features_auctions_v2.merge(subastas_por_dia, how='outer', left_index=True, right_index=True)
    features_auctions_v3 = features_auctions_v3.merge(subastas_por_dia, how='outer', left_index=True, right_index=True)
    features_auctions_v4 = features_auctions_v4.merge(subastas_por_dia, how='outer', left_index=True, right_index=True)
    features_auctions_v5 = features_auctions_v5.merge(subastas_por_dia, how='outer', left_index=True, right_index=True)
    
features_auctions_v1 = features_auctions_v1.fillna(0)
features_auctions_v2 = features_auctions_v2.fillna(0)
features_auctions_v3 = features_auctions_v3.fillna(0)
features_auctions_v4 = features_auctions_v4.fillna(0)
features_auctions_v5 = features_auctions_v5.fillna(0)

In [55]:
features_auctions_v1 = pd.merge(train_st[['device_id']],features_auctions_v1,left_on='device_id',right_on='device_id',how='left')

In [56]:
features_auctions_v1.head()

,device_id,cantidad_de_subastas_en_ventana,cantidad_eventos_Monday,cantidad_eventos_Tuesday,cantidad_eventos_Wednesday,cantidad_eventos_Thursday,cantidad_eventos_Friday,cantidad_eventos_Saturday,cantidad_eventos_Sunday
0,1826643666390887030,188.0,0.0,3.0,0.0,88.0,14.0,94.0,3.0
1,7037174172278258682,404.0,138.0,196.0,149.0,423.0,181.0,178.0,183.0
2,3392065368947589877,624.0,0.0,1.0,2.0,354.0,274.0,0.0,0.0
3,1228982273563226229,1863.0,161.0,93.0,173.0,452.0,1128.0,559.0,107.0
4,4123059034628125459,8.0,1.0,26.0,6.0,12.0,42.0,0.0,0.0


In [57]:
features_auctions_v1.shape

(475338, 9)

In [58]:
features_auctions_v1.fillna(0,inplace=True)

In [59]:
features_auctions_v1.sort_values(by='device_id',inplace=True)

### Features de Events

In [70]:
events_v1 = events.loc[(events['date'] >= pd.to_datetime('2019-04-18')) & (events['date'] <= pd.to_datetime('2019-04-21'))]
events_v2 = events.loc[(events['date'] >= pd.to_datetime('2019-04-19')) & (events['date'] <= pd.to_datetime('2019-04-22'))]
events_v3 = events.loc[(events['date'] >= pd.to_datetime('2019-04-20')) & (events['date'] <= pd.to_datetime('2019-04-23'))]
events_v4 = events.loc[(events['date'] >= pd.to_datetime('2019-04-21')) & (events['date'] <= pd.to_datetime('2019-04-24'))]

ventanas_events = [events_v1, events_v2, events_v3, events_v4]

In [71]:
features_events = []

for ventana in ventanas_events:
    features_ventana = ventana.groupby('ref_hash').agg({'ref_hash':'size'})
    features_ventana.columns = ['cantidad_de_eventos_en_ventana']
    features_events.append(features_ventana)

In [72]:
for index, ventana in enumerate(ventanas_events):
    for dia in Week_Day:
        eventos_por_dia = ventana[ventana['date'].dt.day_name() == dia].groupby('ref_hash').agg({'ref_hash':'size'})
        eventos_por_dia.columns = ['cantidad_eventos_{}'.format(dia)]
        
        features_events[index] = features_events[index].merge(eventos_por_dia, how='outer', left_index=True, right_index=True)
        features_events[index].fillna(0, inplace=True)

In [73]:
features_events_v1 = pd.merge(train_st[['device_id']],features_events[0],left_on='device_id',right_on='ref_hash',how='left')

In [74]:
features_events_v1.head()

,device_id,cantidad_de_eventos_en_ventana,cantidad_eventos_Monday,cantidad_eventos_Tuesday,cantidad_eventos_Wednesday,cantidad_eventos_Thursday,cantidad_eventos_Friday,cantidad_eventos_Saturday,cantidad_eventos_Sunday
0,1826643666390887030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7037174172278258682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3392065368947589877,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
3,1228982273563226229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4123059034628125459,8.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0


In [75]:
features_events_v1.shape

(475338, 9)

In [76]:
features_events_v1.fillna(0,inplace=True)

In [77]:
features_events_v1.sort_values(by='device_id',inplace=True)

#### Eventos por hora

In [78]:
for index, ventana in enumerate(ventanas_events):
    for hora in range(24):
        eventos_por_hora = ventana[ventana['date'].dt.hour == hora].groupby('ref_hash').agg({'ref_hash':'size'})
        eventos_por_hora.columns = ['cantidad_eventos_hora_{}'.format(hora)]
        features_events[index] = features_events[index].merge(eventos_por_hora, how='outer', left_index=True, right_index=True) 
        features_events[index]['cantidad_eventos_hora_{}'.format(hora)].fillna(0, inplace=True)

In [79]:
features_events[0].head()

,cantidad_de_eventos_en_ventana,cantidad_eventos_Monday,cantidad_eventos_Tuesday,cantidad_eventos_Wednesday,cantidad_eventos_Thursday,cantidad_eventos_Friday,cantidad_eventos_Saturday,cantidad_eventos_Sunday,cantidad_eventos_hora_0,cantidad_eventos_hora_1,...,cantidad_eventos_hora_14,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23
ref_hash,,,,,,,,,,,,,,,,,,,,,
41863526108385,88,0.0,0.0,0.0,24.0,21.0,43.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0
161514654074162,8,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
186034136943920,13,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
360710529886978,3,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0
365882020742330,57,0.0,0.0,0.0,0.0,36.0,21.0,0.0,4.0,6.0,...,1.0,0.0,4.0,1.0,8.0,1.0,3.0,0.0,0.0,0.0


### Clicks

In [39]:
#Hay clicks que suceden fuera del período de análisis.
clicks_a_considerar = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-18')) & (clicks['created'] < pd.to_datetime('2019-04-27'))]

In [40]:
clicks_v1 = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-18')) & (clicks['created'] < pd.to_datetime('2019-04-21'))]
clicks_v2 = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-19')) & (clicks['created'] < pd.to_datetime('2019-04-22'))]
clicks_v3 = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-20')) & (clicks['created'] < pd.to_datetime('2019-04-23'))]
clicks_v4 = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-21')) & (clicks['created'] < pd.to_datetime('2019-04-24'))]
clicks_v5 = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-22')) & (clicks['created'] < pd.to_datetime('2019-04-25'))]

features_clicks_v1 = clicks_v1.groupby('ref_hash').agg({'ref_hash':'size'})
features_clicks_v2 = clicks_v2.groupby('ref_hash').agg({'ref_hash':'size'})
features_clicks_v3 = clicks_v3.groupby('ref_hash').agg({'ref_hash':'size'})
features_clicks_v4 = clicks_v4.groupby('ref_hash').agg({'ref_hash':'size'})
features_clicks_v5 = clicks_v5.groupby('ref_hash').agg({'ref_hash':'size'})

In [41]:
features_clicks_v1.columns = ['cantidad_de_clicks_en_ventana']
features_clicks_v2.columns = ['cantidad_de_clicks_en_ventana']
features_clicks_v3.columns = ['cantidad_de_clicks_en_ventana']
features_clicks_v4.columns = ['cantidad_de_clicks_en_ventana']
features_clicks_v5.columns = ['cantidad_de_clicks_en_ventana']

In [42]:
for dia in Week_Day:
    clicks_por_dia = clicks_a_considerar[clicks_a_considerar['created'].dt.day_name() == dia].groupby('ref_hash').agg({'ref_hash':'size'})
    clicks_por_dia.columns = ['cantidad_clicks_{}'.format(dia)]
    
    features_clicks_v1 = features_clicks_v1.merge(clicks_por_dia, how='outer', left_index=True, right_index=True)
    features_clicks_v2 = features_clicks_v2.merge(clicks_por_dia, how='outer', left_index=True, right_index=True)
    features_clicks_v3 = features_clicks_v3.merge(clicks_por_dia, how='outer', left_index=True, right_index=True)
    features_clicks_v4 = features_clicks_v4.merge(clicks_por_dia, how='outer', left_index=True, right_index=True)
    features_clicks_v5 = features_clicks_v4.merge(clicks_por_dia, how='outer', left_index=True, right_index=True)
    
features_clicks_v1 = features_clicks_v1.fillna(0)
features_clicks_v2 = features_clicks_v2.fillna(0)
features_clicks_v3 = features_clicks_v3.fillna(0)
features_clicks_v4 = features_clicks_v4.fillna(0)
features_clicks_v5 = features_clicks_v4.fillna(0)

In [43]:
features_clicks_v1 = pd.merge(train_st[['device_id']],features_clicks_v1,left_on='device_id',right_on='ref_hash',how='left')

In [44]:
features_clicks_v1.head()

,device_id,cantidad_de_clicks_en_ventana,cantidad_clicks_Monday,cantidad_clicks_Tuesday,cantidad_clicks_Wednesday,cantidad_clicks_Thursday,cantidad_clicks_Friday,cantidad_clicks_Saturday,cantidad_clicks_Sunday
0,1826643666390887030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7037174172278258682,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3392065368947589877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1228982273563226229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4123059034628125459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
features_clicks_v1.shape

(475338, 9)

In [46]:
features_clicks_v1.fillna(0,inplace=True)

In [47]:
features_clicks_v1.sort_values(by='device_id',inplace=True)

### Installs

In [61]:
installs_v1 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-18')) & (installs['created'] < pd.to_datetime('2019-04-21'))]
installs_v2 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-19')) & (installs['created'] < pd.to_datetime('2019-04-22'))]
installs_v3 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-20')) & (installs['created'] < pd.to_datetime('2019-04-23'))]
installs_v4 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-21')) & (installs['created'] < pd.to_datetime('2019-04-24'))]
installs_v5 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-22')) & (installs['created'] < pd.to_datetime('2019-04-25'))]

features_installs_v1 = installs_v1.groupby('ref_hash').agg({'ref_hash':'size'})
features_installs_v2 = installs_v2.groupby('ref_hash').agg({'ref_hash':'size'})
features_installs_v3 = installs_v3.groupby('ref_hash').agg({'ref_hash':'size'})
features_installs_v4 = installs_v4.groupby('ref_hash').agg({'ref_hash':'size'})
features_installs_v5 = installs_v5.groupby('ref_hash').agg({'ref_hash':'size'})

In [62]:
features_installs_v1.columns = ['cantidad_de_instalaciones_en_ventana']
features_installs_v2.columns = ['cantidad_de_instalaciones_en_ventana']
features_installs_v3.columns = ['cantidad_de_instalaciones_en_ventana']
features_installs_v4.columns = ['cantidad_de_instalaciones_en_ventana']
features_installs_v5.columns = ['cantidad_de_instalaciones_en_ventana']

In [63]:
for dia in Week_Day:
    instalaciones_por_dia = installs[installs['created'].dt.day_name() == dia].groupby('ref_hash').agg({'ref_hash':'size'})
    instalaciones_por_dia.columns = ['cantidad_installs_{}'.format(dia)]
    
    features_installs_v1 = features_installs_v1.merge(instalaciones_por_dia, how='outer', left_index=True, right_index=True)
    features_installs_v2 = features_installs_v1.merge(instalaciones_por_dia, how='outer', left_index=True, right_index=True)
    features_installs_v3 = features_installs_v1.merge(instalaciones_por_dia, how='outer', left_index=True, right_index=True)
    features_installs_v4 = features_installs_v1.merge(instalaciones_por_dia, how='outer', left_index=True, right_index=True)
    features_installs_v5 = features_installs_v1.merge(instalaciones_por_dia, how='outer', left_index=True, right_index=True)
    
features_installs_v1 = features_installs_v1.fillna(0)
features_installs_v2 = features_installs_v2.fillna(0)
features_installs_v3 = features_installs_v3.fillna(0)
features_installs_v4 = features_installs_v4.fillna(0)
features_installs_v5 = features_installs_v5.fillna(0)

In [64]:
features_installs_v1 = pd.merge(train_st[['device_id']],features_installs_v1,left_on='device_id',right_on='ref_hash',how='left')

In [65]:
features_installs_v1.head()

,device_id,cantidad_de_instalaciones_en_ventana,cantidad_installs_Monday,cantidad_installs_Tuesday,cantidad_installs_Wednesday,cantidad_installs_Thursday,cantidad_installs_Friday,cantidad_installs_Saturday,cantidad_installs_Sunday
0,1826643666390887030,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,7037174172278258682,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3392065368947589877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1228982273563226229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4123059034628125459,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [66]:
features_installs_v1.shape

(475338, 9)

In [67]:
features_installs_v1.fillna(0,inplace=True)

In [68]:
features_installs_v1.sort_values(by='device_id',inplace=True)

# LABEL

In [86]:
label1 = train_st[['device_id','st_v1']]

In [87]:
label1.shape

(475338, 2)

In [100]:
label1['st_v1'].replace(
    to_replace=[0],
    value=tiempo_maximo,
    inplace=True
)

label1['binary'] = np.where(label1['st_v1']>=tiempo_maximo,False,True)
label1.sort_values(by='device_id',inplace=True)

In [101]:
del label1['device_id']
label1 = label1[['binary','st_v1']].to_records(index=False) 

# Algoritmo

In [102]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

estimator = CoxPHSurvivalAnalysis()
estimator.fit(df_features,label1)

CoxPHSurvivalAnalysis(alpha=0, n_iter=100, tol=1e-09, verbose=0)

In [103]:
pd.Series(estimator.coef_, index=df_features.columns)

ref_hash                     -8.357441e-22
cantidad_de_eventos           1.150665e-05
cantidad_eventos_Monday      -3.380476e-04
cantidad_eventos_Tuesday      4.059600e-04
cantidad_eventos_Wednesday   -5.670885e-04
cantidad_eventos_Thursday     2.586518e-04
cantidad_eventos_Friday       6.406885e-05
cantidad_eventos_Saturday     3.038192e-04
cantidad_eventos_Sunday       1.380709e-04
dtype: float64

#### El resultado es un vector de coeficientes, uno para cada variable, donde cada valor corresponde a la relación del registro y evento

In [104]:
prediction = estimator.predict(df_features)

In [105]:
estimator.score(df_features, label1)

0.5021970863822909

# KNN

In [107]:
set_training = pd.merge(train_st[['device_id','st_v1']],df_features, how='inner', left_on='device_id',right_on='ref_hash')

In [108]:
del set_training['device_id']

In [109]:
columns = list(set_training.columns)
columns.remove('st_v1')

In [110]:
from sklearn.model_selection import train_test_split

#TRAIN_TEST_SPLIT
#Split arrays or matrices into random train and test subsets
X_train, X_test, Y_train, Y_test = train_test_split(set_training.loc[:, columns],
                                                    set_training['st_v1'],
                                                    test_size=0.2,
                                                    random_state=43)

In [111]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

k=5 #-->Hiperparametro
neigh = KNeighborsRegressor(n_neighbors = k ,    #Cantidad de vecinos a usar --> K hiperparametro.
                            n_jobs=-1,           #Define la cantidad de procesadores a usar, -1 todos los disponibles.
                            weights='distance',  #Se define el peso como la inversa a la distancia, los mas cercanos tienen mayor influencia.
                            algorithm='brute'    #Fuerza bruta para el algorimo, podría ser KDTree o sino automatico.
                           )
#En weights se puede pasar una funcion definida por el usuario--> Segundo hiperparametro posible de KNN.

In [112]:
neigh.fit(X_train,Y_train) #Fit the model using X as training data and Y as target values

KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
          weights='distance')

In [113]:
predicted = neigh.predict(X_test) #Predict the target for the provided data

In [114]:
neigh.score(X_test, Y_test)

-0.3960931039558926